In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import date_format
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.functions import unix_timestamp, from_unixtime
#import plotly.graph_objs as go
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import shutil
#init_notebook_mode(connected=True)
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.load('/test_dev/mba-code/dataset.csv', format='csv', header='true', inferSchema='true')
df.select("InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate","InvoiceDateWS" ,date_format(from_unixtime(unix_timestamp('InvoiceDateWS', 'mm/dd/yyy')), 'EEEE').alias('weekday'),"CustomerID","Country").write.save("Invoices.parquet", format="parquet")
parquetFile = spark.read.parquet("Invoices.parquet")
# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")

In [17]:
#yield tuple(k.strip() for k in line.split(','))      
from operator import add
from apyori import apriori
import pyspark.sql.functions as f
DescriptionGrp = spark.sql("SELECT distinct InvoiceNo,StockCode FROM parquetFile group by InvoiceNo,StockCode")
print(DescriptionGrp.rdd.take(2))
minSupport = 0.05 * DescriptionGrp.rdd.count()
apr_tem=DescriptionGrp.rdd.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda x,y: x + y).collect()
pandas_mba_df = pd.DataFrame(apr_tem)
mba_list = pandas_mba_df.values.tolist()
#print(pandas_mba_df)
association_rules=apriori(mba_list,min_support=0.5,min_confidence=0.7,min_lift=1.2,min_length=2)
#association_rules=list(association_rules)
print(association_rules)
#rdd = DescriptionGrp.rdd.filter(lambda x: x[0][2] == x[1][2])
#items_tuple = DescriptionGrp.rdd.map(lambda x: (x[0], list(set(x[1]))))
#items_tuple.collect()
#items = DescriptionGrp.rdd.flatMap(lambda line:line.split(","))
#DescriptionGrp.rdd.take(100).groupByKey().collect()
#items_tuple = items.map(lambda x: (x[0], list(set(x[1]))))
#items.collect()
#df_new = DescriptionGrp.toPandas().apply(pd.Series).unstack().reset_index()
#spDF = sqlContext.createDataFrame(df_new)
#print(items_tuple.take(5))
#itemCount = spDF.rdd.map(lambda item:(item, 1)).reduceByKey(add)
#print(itemCount.take(5))
#l1 = itemCount.filter(lambda (i,c): c > minSupport)
#print(l1.take(5))
#l1.take(5)

[Row(InvoiceNo=u'549099', StockCode=u'21908'), Row(InvoiceNo=u'549101', StockCode=u'22586')]
<generator object apriori at 0x7f6202493820>
